In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)






df_enhanced.csv
df_selected_features.csv
V4_classification_lr3.csv


In [ ]:
# 1. Масштабирование исходного набора данных (df_enhanced)

# Определяем числовые и бинарные/OHE признаки
# Исходные числовые признаки (включая добавленные)
numerical_cols_for_scaling_original = [col for col in df_enhanced.columns if col in numerical_features_enhanced]
# Бинарные/OHE признаки, которые не нужно масштабировать
binary_ohe_cols_original = [col for col in df_enhanced.columns if col.startswith('Wilderness_Area') or col.startswith('Soil_Type')]

# Создаем препроцессор для масштабирования числовых признаков
preprocessor_original = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols_for_scaling_original)],
    remainder='passthrough' # Оставляем остальные признаки как есть (бинарные)
)

In [ ]:


# Применяем масштабирование к исходному набору данных (без целевой переменной)
X_original_scaled = preprocessor_original.fit_transform(df_enhanced.drop('Cover_Type', axis=1))

# Получаем названия колонок после преобразования (это сложнее, так как ColumnTransformer меняет порядок)
# Признаки из numerical_cols_for_scaling_original будут первыми, затем остальные.
# Остальные признаки будут в том же порядке, что и в df_enhanced.drop('Cover_Type', axis=1),
# но исключая numerical_cols_for_scaling_original.

# Получаем названия колонок после transform для preprocessor_original
# Сначала идут масштабированные числовые колонки
scaled_feature_names_original = numerical_cols_for_scaling_original
# Затем идут колонки, которые не были преобразованы (remainder='passthrough')
passthrough_feature_names_original = [col for col in df_enhanced.drop('Cover_Type', axis=1).columns if col not in numerical_cols_for_scaling_original]

all_scaled_feature_names_original = scaled_feature_names_original + passthrough_feature_names_original

# Создаем DataFrame с масштабированными данными
df_original_scaled = pd.DataFrame(X_original_scaled, columns=all_scaled_feature_names_original)
df_original_scaled['Cover_Type'] = df_enhanced['Cover_Type'].values # Добавляем целевой признак
print("\n--- Исходный набор данных после Min-Max масштабирования ---")
print(df_original_scaled.head())


# 2. Масштабирование сформированного набора данных (df_selected_features)

# Определяем числовые и бинарные/OHE признаки в df_selected_features
numerical_cols_for_scaling_selected = [col for col in df_selected_features.columns if col in numerical_features_enhanced] # Используем расширенный список численных признаков
binary_ohe_cols_selected = [col for col in df_selected_features.columns if (col.startswith('Wilderness_Area') or col.startswith('Soil_Type')) and col in df_selected_features.columns]


preprocessor_selected = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols_for_scaling_selected)],
    remainder='passthrough'
)

# Применяем масштабирование к сформированному набору данных (без целевой переменной)
X_selected_scaled = preprocessor_selected.fit_transform(df_selected_features.drop('Cover_Type', axis=1))

# Получаем названия колонок после transform для preprocessor_selected
scaled_feature_names_selected = numerical_cols_for_scaling_selected
passthrough_feature_names_selected = [col for col in df_selected_features.drop('Cover_Type', axis=1).columns if col not in numerical_cols_for_scaling_selected]

all_scaled_feature_names_selected = scaled_feature_names_selected + passthrough_feature_names_selected

# Создаем DataFrame с масштабированными данными
df_selected_features_scaled = pd.DataFrame(X_selected_scaled, columns=all_scaled_feature_names_selected)
df_selected_features_scaled['Cover_Type'] = df_selected_features['Cover_Type'].values # Добавляем целевой признак
print("\n--- Сформированный набор данных после Min-Max масштабирования ---")
print(df_selected_features_scaled.head())

# Сохраняем исходные данные без изменений для анализа (они уже в df_enhanced)
# df_original = df_enhanced.copy() - не нужно, df_enhanced уже есть

In [18]:
df_selected_features.to_csv("../datasets/df_selected_features.csv", index=False)
